In [ ]:
import warnings

import mlflow
import mlflow.sklearn
import numpy as np
import sklearn

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet

# MLflow setup

In [ ]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
mlflow.set_experiment('Boston')

# Tracking

In [ ]:
def metrics(actual, pred):
    mse = np.sqrt(mean_squared_error(actual, pred))
    rmse = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    
    return mse, rmse, r2

def load_data():
    house_dataset = datasets.load_boston()
    house_features, house_targets = house_dataset['data'], house_dataset['target']
    
    train_x, test_x, train_y, test_y = train_test_split(house_features, house_targets, random_state=42)
    
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    train_x, train_y, test_x, test_y = load_data()

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
#         mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [ ]:
train(0.5, 0.5)

In [ ]:
train(0.2, 0.2)

In [ ]:
train(0.1, 0.1)

# Reproducing the environment

Reproducing the training environment can happen on multiple levels. The simplest level is keeping track of the requirements through a conda.yaml file. Then specify the entrypoint for this project by creating a `MLproject` file. Take a look at the `MLProject_conda` and the `conda.yaml` files and try it out if your system supports conda.

To try the conda reproduction simply invoke `mlflow run . -P alpha=0.42`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in conda.yaml.

For a more complete reconstruction of the environment it is possible to create a docker image and spin up a container with all dependencies. To do this you will need a Dockerfile and the `MLProject` file. This offers a more complete solution, as it reproduces the entire system.

For the Docker reconstruction you can first build the image as follows:
`docker build -t mlflow-docker-example -f Dockerfile .`

And then you can simply run:
`mlflow run .` 


# Model deployment

Creating a docker image. This will bring in all the necessary dependencies and load the model ready for inference.

`mlflow models build-docker -m mlruns/{experiment_no}/{run_id_hash}/artifacts/model/ -n {docker_image_name}`

Then we can run the container on a different terminal using the appropriate command:

`docker run -p 9000:9000 {docker_image_name}`

To quickly infer we can just pass to the container (using `curl`) a simple command as below:

`curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"],"data":[[18., 2.31, 0., 0.538, 6.575, 65.2, 4.09, 1., 296, 15.3, 396.9, 4.98, 24]]}' http://127.0.0.1:8080/invocations`